# UnQTube2 Setup Guide

This notebook will guide you through the process of setting up [UnQTube2](https://github.com/Sandeepgaddam5432/UnQTube2).

## 1. Clone Repository and Install Dependencies

First, we'll clone the repository from GitHub and install all required packages:

In [ ]:
!git clone https://github.com/Sandeepgaddam5432/UnQTube2.git
%cd UnQTube2
!pip install -q -r requirements.txt

## 2. Configure Cloudflare Tunnel for Remote Access

We'll use Cloudflare Tunnel to create a secure tunnel to expose our local Streamlit server to the internet.
This will give you a public URL to access your UnQTube2 application from anywhere.

In [ ]:
import os
import platform
import subprocess

# Determine the system architecture and download the appropriate cloudflared binary
system = platform.system().lower()
machine = platform.machine().lower()

cloudflared_url = ""
if system == "linux":
    if "arm" in machine or "aarch" in machine:
        cloudflared_url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-arm64"
    else:
        cloudflared_url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"
elif system == "darwin":
    if "arm" in machine or "aarch" in machine:
        cloudflared_url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-darwin-amd64"
    else:
        cloudflared_url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-darwin-amd64"
else:  # Windows
    cloudflared_url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-windows-amd64.exe"

# Download cloudflared
print(f"📥 Downloading cloudflared from {cloudflared_url}...")
!wget -q {cloudflared_url} -O cloudflared
!chmod +x cloudflared
print("✅ Cloudflared downloaded and made executable!")

## 3. Launch Application and Generate Public URL

Now we'll start the Streamlit server and create a Cloudflare tunnel to make it accessible online:

In [ ]:
import subprocess
import time
import re

print("🚀 Starting UnQTube2...")
# Start Streamlit server on port 8501 in the background
streamlit_proc = subprocess.Popen([
    "streamlit", "run", "./webui/Main.py", "--server.port=8501"
], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Wait for the server to initialize
time.sleep(5)

print("🌐 Creating Cloudflare tunnel to expose UnQTube2...")
# Start cloudflared in the background and redirect output to a log file
cloudflared_log = "cloudflared.log"
cloudflared_proc = subprocess.Popen(
    ["./cloudflared", "tunnel", "--url", "http://localhost:8501"],
    stdout=open(cloudflared_log, "w"),
    stderr=subprocess.STDOUT
)

# Wait for the tunnel to be established and extract the URL
time.sleep(5)
public_url = None
retries = 0

while public_url is None and retries < 10:
    with open(cloudflared_log, "r") as f:
        log_content = f.read()
        # Look for the tunnel URL in the log
        url_match = re.search(r'https://[\w-]+\.trycloudflare\.com', log_content)
        if url_match:
            public_url = url_match.group(0)
        else:
            retries += 1
            time.sleep(1)

if public_url:
    print("✅ Deployment complete! Access your UnQTube2 at:")
    print(public_url)
else:
    print("❌ Failed to establish Cloudflare tunnel. Please check the cloudflared.log file.")
    print("You can also try running the tunnel manually with:")
    print("./cloudflared tunnel --url http://localhost:8501")